# 🧠 AI Agents Course – Day 2
## 📌 Notebook: Structured Output

---

### 🚀 Objective:
Make the output of LLM parseable as JSON, predictable and stable for production usage. 

--- 
###   Tasks: 
Let's create a simple Ordering assistant agent that can take orders from customers.  

---

_Hassan Tabbal • Introduction to AI Agents Development Training — All rights reserved_

In [ ]:
from pydantic_ai import Agent,RunContext
from pydantic import BaseModel
from textwrap import dedent # allows us to format string nicely in code, but remove unwanted white space before sending to LLM
from IPython.display import display_pretty
import nest_asyncio # needed to fix an error when running inside Jupyter notebook
nest_asyncio.apply()

model = "google-gla:gemini-2.5-flash"


In [ ]:
from datetime import datetime
from enum import Enum
class Size(str,Enum):
    small = "Small"
    medium = "Medium"
    large = "Large" 
    undefined = "Undefined"
class OrderLine(BaseModel):
    item:str
    size:Size
    qty:int


class Order(BaseModel):
    date:datetime
    lines:list[OrderLine]
    total:float
    isValid:bool
    status_text:str #describes the status of the order and any message that the LLM wants to pass back to user

In [ ]:
system_prompt = dedent(""""
                       You are an expert customer support agent assisting customers in placing online orders.
                       Our online store is called Jimmy's Pizza and we specialize in preparing delicious Pizzas and Italian Pasta. 
                       We also have side dishes and drinks. 
                       when parsing user's input, make sure you identify the item and its quantity. 
                       Example 1: 
                       Customer says : I want to order 3 vegetarian pizza and 2 Orange Juice with one garlic bread. 
                       Parsed order : 
                       vegetarian pizza : 3, Orange juice : 2, garlic bread : 1
                       After parsing the order, you must validate it to make sure all items exist in our menu.
                       Update the order after validation, indicating the validity of the order and your feedback in the status_text field.
                       Always return the validated order. 

                       """)
agent = Agent(  
    model,
    system_prompt=system_prompt,
     output_type=Order 
)


In [ ]:
@agent.tool_plain
def get_Date()->str:
    """Gets the current date and time.

    Args:
        No arguments.

    Returns:
        A string representint the current date and time.
    """
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
@agent.tool_plain
def validate_Order(order:Order)->bool :
    """Checks if the order is valid or not.

    Args:
        order : The order object that we want to check

    Returns:
        A bool value of True or False, True meaning the order is valid. 
    """
    items = [{"item":"Vegetarian Pizza","price":20},{"item":"Pepperoni Pizza","price":22},{"item":"Penne Arabiata","price":15}]
    isValid = False
    for line in order.lines:
        if any(item['item'].lower() == line.item.lower() for item in items) : 
            isValid = True


    return isValid


In [ ]:
result = agent.run_sync("I want 2 Penne Arabiata and 2 medium size vegetarian pizza and 1 Pepsi")  
display_pretty(result.output)

We can also convert the output to json

In [ ]:
import json
result_json = result.output.model_dump_json(indent=4)
print(result_json)

Now check what happens if we provide an invalid order

In [ ]:
invalid_order = agent.run_sync("I want 3 Tacos and 2 Orange Juice")
print(invalid_order.output.model_dump_json(indent=4))